# Proof of Concept Example

## Get the up to date data from the form 

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

class DataGetter():
    """
    An object that has cridentials to read Drive documents.
    """
    def __init__(self, credsFile='client_secret.json'):
        self.scope  = ['https://spreadsheets.google.com/feeds']
        self.creds  = ServiceAccountCredentials.from_json_keyfile_name(credsFile, self.scope)
        self.client = gspread.authorize(self.creds)

    def basicMovementData(self):
        """
        Gets the training data from the first basic movement form and processes
        it into a list of spoken commands and their corresponding labels.
        """
        rows      = self.client.open("MovementTrainingData").sheet1.get_all_records()

        responces = []
        labels    = []
        
        
        for row in rows:
            for i, response in enumerate(list(row.values())[1:]):
                labels.append(i)
                responces.append(response.lower())
                
        return (responces, labels)
    
    def makeLabelForSinario(self, number, numVarients):
        label = np.zeros(numVarients)
        label[number]  = 1.0
        return label
        

## Process the data

The libraries needed for this task are:

In [2]:
import numpy as np
from collections import Counter

In [3]:
(commands, labels) = DataGetter().basicMovementData()

I will use counters to store the count of particular words in different sinarios. 

In [141]:
# total_counts     = Counter()
# sinario_counters = [Counter() for _ in range(max(labels)+1)]

# # for sinario, responses in enumerate(data):
# #     for response in responses:
# #         for word in response.split():
# #             total_counts[word.lower()] +=1
# #             sinario_counters[sinario][word]  +=1

# for i, command in enumerate(commands):
#     for word in command.split():
#         sinario_counters[labels[i]][word] +=1
#         total_counts[word]+=1
        

The most common words in each sinario are:

In [142]:
# for i, counter in enumerate(sinario_counters):
#     print("Sinario " + str(i+1) + ": "     + str(sinario_counters[i].most_common(5)))

### The full vocabulary

As the input layer to the NN will be a tensor containinging a count of every word possible, we ned to know the full vocabulary. 

In [143]:
# vocab      = set(total_counts.keys())
# vocab_size = len(vocab)

In [144]:
# input_layer = np.zeros((1, vocab_size))
# print("The shape of the input layer tensor is: " + str(input_layer.shape))

In [145]:
# wordToIndex = {}
# for i, word in enumerate(vocab):
#     wordToIndex[word] = i

In [146]:
# def fill_input_layer(response: str):
    
#     global input_layer
#     input_layer *=0
    
#     words        = response.split()
#     word_counter = Counter()
    
#     for word in words:
#         word_counter[word] +=1

#     for word, count in word_counter.items():
#         input_layer[0][wordToIndex[word]] = count

In [147]:
# fill_input_layer(commands[0])

## First Attemept

In [8]:
import time
import sys
import numpy as np

class SentimentNetwork:
    def __init__(self, commands, labels, hidden_nodes = 10, output_nodes = 1,learning_rate = 0.1):
        """Create a SentimenNetwork with the given settings
        Args:
            commands(list) - List of commands used for training
            labels(list) - List of POSITIVE/NEGATIVE labels associated with the given reviews
            hidden_nodes(int) - Number of nodes to create in the hidden layer
            learning_rate(float) - Learning rate to use while training
        
        """

        np.random.seed(1)

        self.pre_process_data(commands, labels)
        
        self.init_network(len(self.command_vocab), hidden_nodes, output_nodes, learning_rate)

    def pre_process_data(self, commands, labels):
        
        # Pre process the reviews
        command_vocab = set()
        for command in commands:
            for word in command.split():
                command_vocab.add(word)
        self.command_vocab      = list(command_vocab)
        self.command_vocab_size = len(self.command_vocab)
        
        self.word2index = {}
        for i,word in enumerate(self.command_vocab):
            self.word2index[word] = i
            
        label_vocab = set()
        for label in labels:
            command_vocab.add(label)
            
        self.label_vocab_size = len(label_vocab)
                
        self.label_vocab      = [self.label_vec_from_number(l) for l in label_vocab]
        

    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Store the number of nodes in input, hidden, and output layers.
        self.input_nodes  = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Store the learning rate
        self.learning_rate = learning_rate

        # Initialize weights
        
        # TODO: initialize self.weights_0_1 as a matrix of zeros. These are the weights between
        #       the input layer and the hidden layer.
        
        # I think that the size of the input nodes by the number of hiden nodes in shape
        self.weights_0_1 = np.zeros((self.input_nodes, self.hidden_nodes))
        
        # TODO: initialize self.weights_1_2 as a matrix of random values. 
        #       These are the weights between the hidden layer and the output layer.
        self.weights_1_2 = self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.output_nodes))
        
        # TODO: Create the input layer, a two-dimensional matrix with shape 
        #       1 x input_nodes, with all values initialized to zero
        self.layer_0 = np.zeros((1,input_nodes))
    
    def label_vec_from_number(self, number ):
        label = np.zeros(19) ## TODO
        label[number] = 1
        return label
        
    
        
    def update_input_layer(self,command):
        """
        Args:
            review(string) - the string of the review
        Returns:
            None
        """
        # clear out previous state by resetting the layer to be all 0s
        self.layer_0 *= 0

        for word in command.split(' '):
            if (word in self.word2index.keys()):
                self.layer_0[0][self.word2index[word]] +=1
            
                
    def get_target_for_label(self,label):
        
        return self.label_vec_from_number(label)
    
        
    def sigmoid(self,x):
        return 1.0/(1+(np.e**(-x)))
    
    def sigmoid_output_2_derivative(self,output):
        return output * (1 - output)

    def train(self, training_commands, training_labels, epochs=5):
        assert(len(training_commands) == len(training_labels))
        
        start = time.time()
        
        
        for _ in range(epochs):
        
            correct_so_far = 0

            # loop through all the given reviews and run a forward and backward pass,
            # updating weights for every item
            for i in range(len(training_commands)):

                # TODO: Get the next review and its correct label
                command   = training_commands[i]
                target    = self.get_target_for_label(training_labels[i])

                # -- Forward Pass -- #

                self.update_input_layer(command)

                layer_1_input  = np.matmul(self.layer_0, self.weights_0_1)
                layer_1_output = layer_1_input  # Brief said to not use an activation function on this layer

                layer_2_input   = np.matmul(layer_1_output, self.weights_1_2)
                layer_2_output  = self.sigmoid(layer_2_input)

                # Output error
                layer_2_error = target - layer_2_output 
                layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2_output)


                # Backpropagated error
                layer_1_error = layer_2_delta.dot(self.weights_1_2.T) # errors propagated to the hidden layer
                layer_1_delta = layer_1_error # hidden layer gradients - no nonlinearity so it's the same as the error

                # Update the weights
                self.weights_1_2 += layer_1_output.T.dot(layer_2_delta) * self.learning_rate # update hidden-to-output weights with gradient descent step
                self.weights_0_1 += self.layer_0.T.dot(layer_1_delta) * self.learning_rate # update input-to-hidden weights with gradient descent step


                # Is the output correct?
                if (all(error < 0.5) for error in layer_2_error):
                    correct_so_far +=1




                elapsed_time = float(time.time() - start)
                commands_per_second = i / elapsed_time if elapsed_time > 0 else 0

                sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_commands)))[:4] \
                                 + "% Speed(commands/sec):" + str(commands_per_second)[0:5] \
                                 + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                                 + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
                if(i % 2500 == 0):
                    print("")

    def test(self, testing_commands, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        correct = 0
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_commands)):
            pred = self.run(testing_commands[i])
#             print(pred[0])
#             print(self.label_vec_from_number(testing_labels[i]))

            
            if(np.array_equal(pred[0], self.label_vec_from_number(testing_labels[i]))):
                correct += 1
                print(testing_labels[i])
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            commands_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_commands)))[:4] \
                             + "% Speed(commands/sec):" + str(commands_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, command):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given command.
        """               
        self.update_input_layer(command.lower())
            
        hidden_input  = np.matmul(self.layer_0, self.weights_0_1)
        hidden_output = hidden_input  # Brief said to not use an activation function on this layer
           
        final_input   = np.matmul(hidden_output, self.weights_1_2)
        final_output  = self.sigmoid(final_input)
        

        
        
        # This is  stupid way of rounding
        for i, val in enumerate(final_output[0]):
            if val < 0.5:
                final_output[0][i] = 0
            else:
                final_output[0][i] = 1

#             if (val - int(val) >= 0.5):
#                 final_output[0][i] =  int (val) + 1
#             else:
#                 final_output[0][i] =  int (val)
            
        return final_output

In [9]:
mlp = SentimentNetwork(commands[:-50],labels[:-50],hidden_nodes=100,output_nodes=19, learning_rate=0.3)

In [10]:
mlp.train(commands[:-50],labels[:-50])

<class 'numpy.ndarray'>
Progress:0.0% Speed(commands/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
<class 'numpy.ndarray'>
Progress:0.10% Speed(commands/sec):119.2 #Correct:2 #Trained:2 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:0.20% Speed(commands/sec):208.6 #Correct:3 #Trained:3 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:0.30% Speed(commands/sec):291.4 #Correct:4 #Trained:4 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:0.40% Speed(commands/sec):198.0 #Correct:5 #Trained:5 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:0.51% Speed(commands/sec):220.6 #Correct:6 #Trained:6 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:0.61% Speed(commands/sec):181.4 #Correct:7 #Trained:7 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:0.71% Speed(commands/sec):126.3 #Correct:8 #Trained:8 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:0.81% Speed(commands/sec):139.4 #Correct:9 #Trained:9 Training Accuracy:100.%<class 'nump

Progress:8.60% Speed(commands/sec):419.4 #Correct:85 #Trained:85 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:8.70% Speed(commands/sec):419.8 #Correct:86 #Trained:86 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:8.81% Speed(commands/sec):418.0 #Correct:87 #Trained:87 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:8.91% Speed(commands/sec):420.3 #Correct:88 #Trained:88 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:9.01% Speed(commands/sec):424.0 #Correct:89 #Trained:89 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:9.11% Speed(commands/sec):422.2 #Correct:90 #Trained:90 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:9.22% Speed(commands/sec):425.6 #Correct:91 #Trained:91 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:9.32% Speed(commands/sec):409.7 #Correct:92 #Trained:92 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:9.42% Speed(commands/sec):395.5 #Correct:93 #Trained:93 Training Accuracy:100.%<class 'numpy.nd

Progress:20.3% Speed(commands/sec):495.9 #Correct:200 #Trained:200 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:20.4% Speed(commands/sec):495.7 #Correct:201 #Trained:201 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:20.5% Speed(commands/sec):494.5 #Correct:202 #Trained:202 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:20.6% Speed(commands/sec):495.2 #Correct:203 #Trained:203 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:20.7% Speed(commands/sec):494.8 #Correct:204 #Trained:204 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:20.9% Speed(commands/sec):494.0 #Correct:205 #Trained:205 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:21.0% Speed(commands/sec):472.2 #Correct:206 #Trained:206 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:21.1% Speed(commands/sec):474.0 #Correct:207 #Trained:207 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:21.2% Speed(commands/sec):475.8 #Correct:208 #Trained:208 Training Accuracy:100

Progress:32.7% Speed(commands/sec):397.6 #Correct:321 #Trained:321 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:32.8% Speed(commands/sec):396.8 #Correct:322 #Trained:322 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:32.9% Speed(commands/sec):397.2 #Correct:323 #Trained:323 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:33.0% Speed(commands/sec):397.7 #Correct:324 #Trained:324 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:33.1% Speed(commands/sec):397.5 #Correct:325 #Trained:325 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:33.2% Speed(commands/sec):391.4 #Correct:326 #Trained:326 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:33.4% Speed(commands/sec):390.6 #Correct:327 #Trained:327 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:33.5% Speed(commands/sec):389.9 #Correct:328 #Trained:328 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:33.6% Speed(commands/sec):389.3 #Correct:329 #Trained:329 Training Accuracy:100

Progress:55.9% Speed(commands/sec):540.6 #Correct:547 #Trained:547 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.0% Speed(commands/sec):538.5 #Correct:548 #Trained:548 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.1% Speed(commands/sec):538.9 #Correct:549 #Trained:549 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.2% Speed(commands/sec):539.1 #Correct:550 #Trained:550 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.3% Speed(commands/sec):539.4 #Correct:551 #Trained:551 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.4% Speed(commands/sec):539.9 #Correct:552 #Trained:552 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.5% Speed(commands/sec):540.4 #Correct:553 #Trained:553 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.6% Speed(commands/sec):541.1 #Correct:554 #Trained:554 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:56.7% Speed(commands/sec):541.7 #Correct:555 #Trained:555 Training Accuracy:100

...

Progress:27.1% Speed(commands/sec):48.02 #Correct:266 #Trained:266 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.2% Speed(commands/sec):48.14 #Correct:267 #Trained:267 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.3% Speed(commands/sec):48.16 #Correct:268 #Trained:268 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.4% Speed(commands/sec):48.28 #Correct:269 #Trained:269 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.5% Speed(commands/sec):48.45 #Correct:270 #Trained:270 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.6% Speed(commands/sec):48.61 #Correct:271 #Trained:271 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.7% Speed(commands/sec):48.75 #Correct:272 #Trained:272 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.8% Speed(commands/sec):48.92 #Correct:273 #Trained:273 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:27.9% Speed(commands/sec):48.97 #Correct:274 #Trained:274 Training Accuracy:100

Progress:47.5% Speed(commands/sec):80.99 #Correct:465 #Trained:465 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:47.6% Speed(commands/sec):81.15 #Correct:466 #Trained:466 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:47.7% Speed(commands/sec):81.24 #Correct:467 #Trained:467 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:47.8% Speed(commands/sec):81.21 #Correct:468 #Trained:468 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:47.9% Speed(commands/sec):81.27 #Correct:469 #Trained:469 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:48.0% Speed(commands/sec):81.43 #Correct:470 #Trained:470 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:48.1% Speed(commands/sec):81.55 #Correct:471 #Trained:471 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:48.2% Speed(commands/sec):81.72 #Correct:472 #Trained:472 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:48.3% Speed(commands/sec):81.85 #Correct:473 #Trained:473 Training Accuracy:100

Progress:61.8% Speed(commands/sec):101.8 #Correct:605 #Trained:605 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:61.9% Speed(commands/sec):101.8 #Correct:606 #Trained:606 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:62.0% Speed(commands/sec):101.7 #Correct:607 #Trained:607 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:62.1% Speed(commands/sec):101.8 #Correct:608 #Trained:608 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:62.2% Speed(commands/sec):102.0 #Correct:609 #Trained:609 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:62.3% Speed(commands/sec):102.1 #Correct:610 #Trained:610 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:62.5% Speed(commands/sec):102.3 #Correct:611 #Trained:611 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:62.6% Speed(commands/sec):102.4 #Correct:612 #Trained:612 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:62.7% Speed(commands/sec):102.5 #Correct:613 #Trained:613 Training Accuracy:100

Progress:84.0% Speed(commands/sec):133.6 #Correct:821 #Trained:821 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.1% Speed(commands/sec):133.7 #Correct:822 #Trained:822 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.2% Speed(commands/sec):133.7 #Correct:823 #Trained:823 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.3% Speed(commands/sec):133.5 #Correct:824 #Trained:824 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.4% Speed(commands/sec):133.7 #Correct:825 #Trained:825 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.5% Speed(commands/sec):133.7 #Correct:826 #Trained:826 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.6% Speed(commands/sec):133.9 #Correct:827 #Trained:827 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.7% Speed(commands/sec):134.0 #Correct:828 #Trained:828 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:84.8% Speed(commands/sec):134.1 #Correct:829 #Trained:829 Training Accuracy:100

Progress:97.9% Speed(commands/sec):150.8 #Correct:957 #Trained:957 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.0% Speed(commands/sec):150.8 #Correct:958 #Trained:958 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.1% Speed(commands/sec):150.6 #Correct:959 #Trained:959 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.2% Speed(commands/sec):150.7 #Correct:960 #Trained:960 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.3% Speed(commands/sec):150.8 #Correct:961 #Trained:961 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.4% Speed(commands/sec):150.9 #Correct:962 #Trained:962 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.5% Speed(commands/sec):151.1 #Correct:963 #Trained:963 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.6% Speed(commands/sec):151.1 #Correct:964 #Trained:964 Training Accuracy:100.%<class 'numpy.ndarray'>
Progress:98.7% Speed(commands/sec):151.3 #Correct:965 #Trained:965 Training Accuracy:100

In [7]:
mlp.test(commands[-50:],labels[-50:])

Progress:6.0% Speed(commands/sec):113.3 #Correct:0 #Tested:4 Testing Accuracy:0.0%11
Progress:16.0% Speed(commands/sec):261.2 #Correct:1 #Tested:9 Testing Accuracy:11.1%16
Progress:18.0% Speed(commands/sec):289.8 #Correct:2 #Tested:10 Testing Accuracy:20.0%17
Progress:44.0% Speed(commands/sec):472.5 #Correct:3 #Tested:23 Testing Accuracy:13.0%11
Progress:66.0% Speed(commands/sec):511.7 #Correct:4 #Tested:34 Testing Accuracy:11.7%3
Progress:72.0% Speed(commands/sec):553.3 #Correct:5 #Tested:37 Testing Accuracy:13.5%6
Progress:82.0% Speed(commands/sec):603.1 #Correct:6 #Tested:42 Testing Accuracy:14.2%11
Progress:94.0% Speed(commands/sec):488.8 #Correct:7 #Tested:48 Testing Accuracy:14.5%17
Progress:98.0% Speed(commands/sec):491.6 #Correct:8 #Tested:50 Testing Accuracy:16.0%